In [68]:

import pandas as pd
import fastf1 as ff1
import fastf1.plotting
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
import warnings
import re
import datetime
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.options.display.max_rows = 75
pd.options.display.max_columns = 99

fastf1.Cache.enable_cache('../cache')
fastf1.plotting.setup_mpl(mpl_timedelta_support=True, color_scheme=None, misc_mpl_mods=False)

In [69]:
gp_round = 6
url = f'https://f1-jedha-bucket.s3.eu-west-3.amazonaws.com/data/full_data-round_{gp_round}.csv'
dict_nb_laps = {1: 57, 2: 50, 3: 58, 4: 63, 5: 57, 6: 66, 7: 78, 8: 51, 9: 70, 10: 52, 11: 71, 12: 53, 13: 70, 14: 44, 15: 72, 16: 53, 17: 61, 18: 53, 19: 56, 20: 71, 21: 71, 22: 58}
nb_laps = dict_nb_laps[gp_round]

In [70]:
dffp = pd.read_csv(url, index_col=0)


In [71]:
dffp = dffp.drop(dffp.columns[[0,5,6,7,8,9,10,11,12,13,14,15,16,17,21,25,26]], axis=1)

In [72]:
dffp_temp = dffp.copy()
dffp_temp['LapTime'] = dffp_temp['LapTime'].apply(pd.to_timedelta)
dffp_times = dffp_temp.groupby('Compound')['LapTime'].mean()
dffp_times = pd.DataFrame(dffp_times).sort_values(by='LapTime').reset_index()
dffp_times

,Compound,LapTime
0,UNKNOWN,0 days 00:01:21.326000
1,SOFT,0 days 00:01:22.017585987
2,MEDIUM,0 days 00:01:23.137373333
3,HARD,0 days 00:01:24.142683544


In [73]:
hard_to_soft = (dffp_times['LapTime'].iloc[2] - dffp_times['LapTime'].iloc[0]) * 0.75
hard_to_soft

Timedelta('0 days 00:00:01.358529999')

In [74]:
medium_to_soft = (dffp_times['LapTime'].iloc[1] - dffp_times['LapTime'].iloc[0]) * 0.75
medium_to_soft

Timedelta('0 days 00:00:00.518689490')

In [75]:
dffp = dffp[dffp['TyreLife'].notna()]

In [76]:
dffp_copy = dffp.copy()
dffp_copy['LapTime'] = dffp['LapTime'].apply(pd.to_timedelta)

In [77]:
def time_to_seconds(string):
    array = re.findall(r'[0-9]+', str(string))
    array = array[1:]
    array[0] = int(array[0]) * 3600
    array[1] = int(array[1]) * 60
    try : 
        array = float(str(array[0] + array[1] + int(array[2])) + "." + array[3])
        return array
    except :
        return float(str(int(array[-2]) + int(array[-1])) + ".0")

dffp_copy["LapTimeSeconds"] = dffp_copy["LapTime"].apply(time_to_seconds)


In [78]:
compound_colors = {
    'SOFT': '#FF3333',
    'MEDIUM': '#FFF200',
    'HARD': '#EBEBEB',
}

In [79]:
dffp_full = dffp_copy.groupby(['TyreLife','Compound'] ,as_index=False)[['LapTimeSeconds']].mean().sort_values(by= ['Compound','TyreLife'])
dffp_full

,TyreLife,Compound,LapTimeSeconds
0,2.0,HARD,84.766333
4,3.0,HARD,84.286875
7,4.0,HARD,84.145333
10,5.0,HARD,84.141000
13,6.0,HARD,84.134000
16,7.0,HARD,84.195429
19,8.0,HARD,84.076125
22,9.0,HARD,84.212125
25,10.0,HARD,84.169500
27,11.0,HARD,84.500833


In [80]:
# dffp_filtered = dffp_full.sort_values(by= ['Compound','TyreLife'])
dffp_filtered = dffp_full[dffp_full['TyreLife'] > 5.0]
dffp_filtered

,TyreLife,Compound,LapTimeSeconds
13,6.0,HARD,84.134000
16,7.0,HARD,84.195429
19,8.0,HARD,84.076125
22,9.0,HARD,84.212125
25,10.0,HARD,84.169500
27,11.0,HARD,84.500833
30,12.0,HARD,83.695250
32,13.0,HARD,84.795500
33,14.0,HARD,84.613000
34,15.0,HARD,84.798000


In [81]:
dffp_soft = dffp_filtered[dffp_filtered['Compound'] == 'SOFT'].reset_index(drop=True)
dffp_medium = dffp_filtered[dffp_filtered['Compound'] == 'MEDIUM'].reset_index(drop=True)
dffp_hard = dffp_filtered[dffp_filtered['Compound'] == 'HARD'].reset_index(drop=True)

In [82]:
dffp_soft['LapTimeSeconds'].min()

81.83833333333332

In [83]:
dffp_soft['LapTimeSeconds'].iloc[-1] - dffp_soft['LapTimeSeconds'].iloc[0]

3.506666666666675

In [84]:
diff_soft = (dffp_soft['LapTimeSeconds'].iloc[-1] - dffp_soft['LapTimeSeconds'].iloc[0])
diff_medium = (dffp_medium['LapTimeSeconds'].iloc[-1] - dffp_medium['LapTimeSeconds'].iloc[0])
diff_hard = dffp_hard['LapTimeSeconds'].iloc[-1] - dffp_hard['LapTimeSeconds'].iloc[0]

In [85]:
if 'WET' in dffp_filtered['Compound'].tolist() or 'INTERDMEDIATE' in dffp_filtered['Compound'].tolist() or diff_soft < 0 or diff_medium < 0 or diff_hard < 0:
    diff_soft_per_lap = 0.25
    diff_medium_per_lap = 0.15
    diff_hard_per_lap = 0.09
    
else:
    diff_soft_per_lap = diff_soft / len(dffp_soft)
    diff_medium_per_lap = diff_medium / len(dffp_medium)
    diff_hard_per_lap = diff_hard / len(dffp_hard)

In [86]:
print(diff_soft_per_lap)
print(diff_medium_per_lap)
print(diff_hard_per_lap)

0.25
0.15
0.09


In [87]:
fuel_per_lap_kg = 105 / nb_laps
mean_laps = 60
time_gain_per_kg = 0.035
laps_coeff = mean_laps / nb_laps
time_gained_per_lap = laps_coeff * time_gain_per_kg * fuel_per_lap_kg
time_gained_per_lap

0.050619834710743807

In [88]:
dffp_soft['FuelGain'] = time_gained_per_lap
dffp_soft['FuelGain'] = dffp_soft['FuelGain'].cumsum()
dffp_soft['AdjustedTime'] = dffp_soft['LapTimeSeconds'] + dffp_soft['FuelGain']

In [89]:
laptime_min_series = dffp_full[dffp_full['Compound'] == 'SOFT'].sort_values('LapTimeSeconds', ascending=True).iloc[0]
laptime_min = {'TyreLife': laptime_min_series['TyreLife'], 'LapTimeSeconds': laptime_min_series['LapTimeSeconds']}
laptime_min

{'TyreLife': 3.0, 'LapTimeSeconds': 81.06035294117648}

In [90]:
laptime_min_calc_soft =  (laptime_min['LapTimeSeconds'] - ((laptime_min['TyreLife'] - 1) * diff_soft_per_lap)) + (nb_laps * time_gained_per_lap)
laptime_min_calc_soft

83.90126203208557

In [91]:
laptime_min_calc_medium = (laptime_min_calc_soft + time_to_seconds(medium_to_soft))
laptime_min_calc_medium

84.41995152208557

In [92]:
laptime_min_calc_hard = laptime_min_calc_soft + time_to_seconds(hard_to_soft)
laptime_min_calc_hard

85.25979203108557

In [93]:
def get_deg_values(dc, dr):
    list_times = []

    for i in range(1, nb_laps + 1):
        dr = dr * (1 + dc)
        list_times.append(dr)
        
    return list_times

list_times_soft = get_deg_values(0.13, diff_soft_per_lap)
list_times_medium = get_deg_values(0.085, diff_medium_per_lap)
list_times_hard = get_deg_values(0.065, diff_medium_per_lap)

In [94]:
df_times_soft = pd.DataFrame(columns=['Lap', 'Tyre', 'LapTimeSeconds', 'DeltaDeg', 'FuelDeg', 'AdjustedTime'])



df_times_soft['Lap'] = list(range(1, nb_laps + 1))
df_times_soft['Tyre'] = 'SOFT'
df_times_soft['LapTimeSeconds'] = laptime_min_calc_soft
df_times_soft['DeltaDeg'] = list_times_soft
df_times_soft['FuelDeg'] = time_gained_per_lap * df_times_soft['Lap']
df_times_soft['AdjustedTime'] = df_times_soft.apply(lambda row: row['LapTimeSeconds'] + row['DeltaDeg'], axis=1)
df_times_soft['FinalLapTime'] = df_times_soft.apply(lambda row: row['AdjustedTime'] - row['FuelDeg'], axis=1)


In [95]:
df_times_medium = pd.DataFrame(columns=['Lap', 'Tyre', 'LapTimeSeconds', 'DeltaDeg', 'FuelDeg', 'AdjustedTime'])

df_times_medium['Lap'] = list(range(1, nb_laps + 1))
df_times_medium['Tyre'] = 'MEDIUM'
df_times_medium['LapTimeSeconds'] = laptime_min_calc_medium
df_times_medium['DeltaDeg'] = list_times_medium
df_times_medium['FuelDeg'] = time_gained_per_lap * df_times_medium['Lap']
df_times_medium['AdjustedTime'] = df_times_medium.apply(lambda row: row['LapTimeSeconds'] + row['DeltaDeg'], axis=1)
df_times_medium['FinalLapTime'] = df_times_medium.apply(lambda row: row['AdjustedTime'] - row['FuelDeg'], axis=1)


df_times_medium

,Lap,Tyre,LapTimeSeconds,DeltaDeg,FuelDeg,AdjustedTime,FinalLapTime
0,1,MEDIUM,84.419952,0.162750,0.050620,84.582702,84.532082
1,2,MEDIUM,84.419952,0.176584,0.101240,84.596535,84.495296
2,3,MEDIUM,84.419952,0.191593,0.151860,84.611545,84.459685
3,4,MEDIUM,84.419952,0.207879,0.202479,84.627830,84.425351
4,5,MEDIUM,84.419952,0.225549,0.253099,84.645500,84.392401
5,6,MEDIUM,84.419952,0.244720,0.303719,84.664672,84.360953
6,7,MEDIUM,84.419952,0.265521,0.354339,84.685473,84.331134
7,8,MEDIUM,84.419952,0.288091,0.404959,84.708042,84.303083
8,9,MEDIUM,84.419952,0.312578,0.455579,84.732530,84.276951
9,10,MEDIUM,84.419952,0.339148,0.506198,84.759099,84.252901


In [96]:
df_times_hard = pd.DataFrame(columns=['Lap', 'Tyre', 'LapTimeSeconds', 'DeltaDeg', 'FuelDeg', 'AdjustedTime'])

df_times_hard['Lap'] = list(range(1, nb_laps + 1))
df_times_hard['Tyre'] = 'HARD'
df_times_hard['LapTimeSeconds'] = laptime_min_calc_hard
df_times_hard['DeltaDeg'] = list_times_hard
df_times_hard['FuelDeg'] = time_gained_per_lap * df_times_hard['Lap']
df_times_hard['AdjustedTime'] = df_times_hard.apply(lambda row: row['LapTimeSeconds'] + row['DeltaDeg'], axis=1)
df_times_hard['FinalLapTime'] = df_times_hard.apply(lambda row: row['AdjustedTime'] - row['FuelDeg'], axis=1)


df_times_hard

,Lap,Tyre,LapTimeSeconds,DeltaDeg,FuelDeg,AdjustedTime,FinalLapTime
0,1,HARD,85.259792,0.159750,0.050620,85.419542,85.368922
1,2,HARD,85.259792,0.170134,0.101240,85.429926,85.328686
2,3,HARD,85.259792,0.181192,0.151860,85.440984,85.289125
3,4,HARD,85.259792,0.192970,0.202479,85.452762,85.250283
4,5,HARD,85.259792,0.205513,0.253099,85.465305,85.212206
5,6,HARD,85.259792,0.218871,0.303719,85.478663,85.174944
6,7,HARD,85.259792,0.233098,0.354339,85.492890,85.138551
7,8,HARD,85.259792,0.248249,0.404959,85.508041,85.103083
8,9,HARD,85.259792,0.264386,0.455579,85.524178,85.068599
9,10,HARD,85.259792,0.281571,0.506198,85.541363,85.035164


In [97]:
df_times = pd.concat([df_times_soft, df_times_medium, df_times_hard])
df_times['TimeStr'] = df_times['FinalLapTime'].apply(lambda x: str(datetime.timedelta(seconds=x))[2:-3])
df_times

,Lap,Tyre,LapTimeSeconds,DeltaDeg,FuelDeg,AdjustedTime,FinalLapTime,TimeStr
0,1,SOFT,83.901262,0.282500,0.050620,84.183762,84.133142,01:24.133
1,2,SOFT,83.901262,0.319225,0.101240,84.220487,84.119247,01:24.119
2,3,SOFT,83.901262,0.360724,0.151860,84.261986,84.110127,01:24.110
3,4,SOFT,83.901262,0.407618,0.202479,84.308880,84.106401,01:24.106
4,5,SOFT,83.901262,0.460609,0.253099,84.361871,84.108772,01:24.108
...,...,...,...,...,...,...,...,...
61,62,HARD,85.259792,7.443324,3.138430,92.703116,89.564687,01:29.564
62,63,HARD,85.259792,7.927140,3.189050,93.186932,89.997883,01:29.997
63,64,HARD,85.259792,8.442405,3.239669,93.702197,90.462527,01:30.462
64,65,HARD,85.259792,8.991161,3.290289,94.250953,90.960664,01:30.960


In [98]:
hovertemplate = '<b>Lap:</b> %{x}<br><b>Time:</b> %{customdata}'

fig = go.Figure()
fig.add_trace(go.Scatter(x=df_times[df_times['Tyre'] == 'SOFT']['Lap'],
                         y=df_times[df_times['Tyre'] == 'SOFT']['FinalLapTime'],
                         mode='lines',
                         name='Soft',
                         line_color=compound_colors['SOFT'],
                         customdata=df_times[df_times['Tyre'] == 'SOFT']['TimeStr'],
                         hovertemplate=hovertemplate))

fig.add_trace(go.Scatter(x=df_times[df_times['Tyre'] == 'MEDIUM']['Lap'],
                         y=df_times[df_times['Tyre'] == 'MEDIUM']['FinalLapTime'],
                         mode='lines',
                         name='Medium',
                         line_color=compound_colors['MEDIUM'],
                         customdata=df_times[df_times['Tyre'] == 'MEDIUM']['TimeStr'],
                         hovertemplate=hovertemplate))

fig.add_trace(go.Scatter(x=df_times[df_times['Tyre'] == 'HARD']['Lap'],
                         y=df_times[df_times['Tyre'] == 'HARD']['FinalLapTime'],
                         mode='lines',
                         name='Hard',
                         line_color=compound_colors['HARD'],
                         customdata=df_times[df_times['Tyre'] == 'HARD']['TimeStr'],
                         hovertemplate=hovertemplate))

fig.update_layout(width=1000, height=700, template='plotly_dark', yaxis_range=[df_times['FinalLapTime'].min() - 2, df_times['LapTimeSeconds'].max() + 10], hovermode='x')
fig.show()

In [99]:
df_times.to_csv(f'../data/prediction_times_round_{gp_round}.csv')